In [5]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import nibabel as nib

from tensorflow import keras
from tensorflow.keras import layers

from scipy import ndimage

In [6]:
scan_paths = [
    os.path.join(os.getcwd(), "data/BET_BSE_DATA", x)
    for x in os.listdir("data/BET_BSE_DATA")
]

print("Brain scans: " + str(len(scan_paths)))

Brain scans: 3


In [7]:
def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load(filepath)
    # Get raw data
    scan = scan.get_fdata()
    return scan


def normalize(volume):
    """Normalize the volume"""
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume


def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Rotate
    img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img


def process_scan(path):
    """Read and resize volume"""
    # Read scan
    volume = read_nifti_file(path)
    # Normalize
    volume = normalize(volume)
    # Resize width, height and depth
    volume = resize_volume(volume)
    return volume

In [8]:
import csv
 
# csv file name
filename = "data/BET_BSE_DATA/Label_file.csv"
 
# initializing the titles and rows list
fields = []
labels_facical_feature_dict = dict()
labels_brain_feature_dict = dict()
 
# reading csv file
with open(filename, 'r') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
     
    # extracting field names through first row
    fields = next(csvreader)
 
    # extracting each data row one by one
    for row in csvreader:
        #print(row)
        if row[1].lower() == 'yes':
            labels_facical_feature_dict[row[0]+'.gz'] = 1
        else:
            labels_facical_feature_dict[row[0]+'.gz'] = 0
        if row[2].lower() == 'yes':
            labels_brain_feature_dict[row[0]+'.gz'] = 1
        else:
            labels_brain_feature_dict[row[0]+'.gz'] = 0
 
    # get total number of rows
    print("Total no. of rows: %d"%(csvreader.line_num))

#printing the field names
print('Field names are:' + ', '.join(field for field in fields))
#print(labels_dict)

Total no. of rows: 1312
Field names are:Filename, Recognizable-Facial-Feature, Brain-Feature-Loss


In [49]:
brain_scans=[]
labels_facial_feature=[]
labels_brain_feature=[]
labels_to_file=[]
for path in os.listdir("data/BET_BSE_DATA/files"):
    #brain_scans.append(process_scan("/home/joelkik/DataMining/BET_BSE_DATA/files/"+path))
    labels_facial_feature.append(labels_facical_feature_dict[path])
    labels_brain_feature.append(labels_brain_feature_dict[path])
    labels_to_file.append(path)

In [50]:
#brain_scans_np = np.array(brain_scans)
labels_facial_feature_np = np.array(labels_facial_feature)
labels_brain_feature_np = np.array(labels_brain_feature)
brain_scans_np=np.load('brain_scans_np.npy')

In [31]:
def get_model(width=128, height=128, depth=64):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model


# Build model.
model = get_model(width=128, height=128, depth=64)
model.summary()

Model: "3dcnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 64, 1)  0         
                             ]                                   
                                                                 
 conv3d_4 (Conv3D)           (None, 126, 126, 62, 64)  1792      
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 63, 63, 31, 64)   0         
 3D)                                                             
                                                                 
 batch_normalization_4 (Batc  (None, 63, 63, 31, 64)   256       
 hNormalization)                                                 
                                                                 
 conv3d_5 (Conv3D)           (None, 61, 61, 29, 64)    110656    
                                                             

In [45]:
# model = keras.models.load_model("3d_brain_image_classification.h5")
model.load_weights("3d_brain_image_classification.h5")

In [46]:
len(labels_facial_feature_np)

1311

In [36]:
for i in range(10):
    prediction = model.predict(np.expand_dims(brain_scans_np[i], axis=0))[0]
    print("Scan File: ", labels_to_file[i], ", Actual: ", labels_facial_feature_np[i], ", Predicted: ",round(float(prediction)))

Scan File:  IXI369-Guys-0924-T1_bse_high_s75_r2.nii.gz , Actual:  0 , Predicted:  0
Scan File:  IXI270-Guys-0847-T1_bet_88.nii.gz , Actual:  0 , Predicted:  0
Scan File:  IXI333-IOP-0926-T1_bse_default.nii.gz , Actual:  1 , Predicted:  1
Scan File:  IXI483-HH-2177-T1_bet_12.nii.gz , Actual:  1 , Predicted:  1
Scan File:  IXI464-IOP-1029-T1_bet_08.nii.gz , Actual:  1 , Predicted:  1
Scan File:  IXI205-HH-1649-T1_bet_73.nii.gz , Actual:  0 , Predicted:  0
Scan File:  IXI544-HH-2395-T1_bse_less_s51_r1.nii.gz , Actual:  1 , Predicted:  1
Scan File:  IXI033-HH-1259-T1_bet_76.nii.gz , Actual:  0 , Predicted:  0
Scan File:  IXI260-HH-1805-T1_bet_12.nii.gz , Actual:  1 , Predicted:  1
Scan File:  IXI241-Guys-0833-T1_bse_high_s74_r2.nii.gz , Actual:  0 , Predicted:  0


In [47]:
y_preds_ff = []
for i in range(len(labels_facial_feature_np)):
    prediction = model.predict(np.expand_dims(brain_scans_np[i], axis=0))[0]
    y_preds_ff.append(round(float(prediction)))

In [48]:
model = keras.models.load_model("3d_brain_feature_loss_model.h5")

In [39]:
for i in range(len(10)):
    prediction = model.predict(np.expand_dims(brain_scans_np[i], axis=0))[0]
    print("Scan File: ", labels_to_file[i], ", Actual: ", labels_brain_feature[i], ", Predicted: ",round(float(prediction)))

Scan File:  IXI369-Guys-0924-T1_bse_high_s75_r2.nii.gz , Actual:  1 , Predicted:  1
Scan File:  IXI270-Guys-0847-T1_bet_88.nii.gz , Actual:  1 , Predicted:  1
Scan File:  IXI333-IOP-0926-T1_bse_default.nii.gz , Actual:  0 , Predicted:  0
Scan File:  IXI483-HH-2177-T1_bet_12.nii.gz , Actual:  0 , Predicted:  0
Scan File:  IXI464-IOP-1029-T1_bet_08.nii.gz , Actual:  0 , Predicted:  0
Scan File:  IXI205-HH-1649-T1_bet_73.nii.gz , Actual:  1 , Predicted:  1
Scan File:  IXI544-HH-2395-T1_bse_less_s51_r1.nii.gz , Actual:  0 , Predicted:  0
Scan File:  IXI033-HH-1259-T1_bet_76.nii.gz , Actual:  1 , Predicted:  1
Scan File:  IXI260-HH-1805-T1_bet_12.nii.gz , Actual:  0 , Predicted:  0
Scan File:  IXI241-Guys-0833-T1_bse_high_s74_r2.nii.gz , Actual:  1 , Predicted:  1
Scan File:  IXI569-Guys-1101-T1_bse_high_s75_r2.nii.gz , Actual:  1 , Predicted:  1
Scan File:  IXI635-HH-2691-T1_bet_12.nii.gz , Actual:  0 , Predicted:  0
Scan File:  IXI639-Guys-1088-T1_bet_85.nii.gz , Actual:  1 , Predicted:  

In [51]:
y_preds_bf = []
for i in range(len(labels_brain_feature_np)):
    prediction = model.predict(np.expand_dims(brain_scans_np[i], axis=0))[0]
    y_preds_bf.append(round(float(prediction)))

In [19]:
brain_scans_np.shape

(1311, 128, 128, 64)

In [24]:
brain_scans_np[0].shape

(128, 128, 64)

In [30]:
brain_scans_np[0][0][0]

array([0.71428573, 0.71428573, 0.71428573, 0.71428573, 0.71428573,
       0.71428573, 0.71428573, 0.71428573, 0.71428573, 0.71428573,
       0.71428573, 0.71428573, 0.71428573, 0.71428573, 0.71428573,
       0.71428573, 0.71428573, 0.71428573, 0.71428573, 0.71428573,
       0.71428573, 0.71428573, 0.71428573, 0.71428573, 0.71428573,
       0.71428573, 0.71428573, 0.71428573, 0.71428573, 0.71428573,
       0.71428573, 0.71428573, 0.71428573, 0.71428573, 0.71428573,
       0.71428573, 0.71428573, 0.71428573, 0.71428573, 0.71428573,
       0.71428573, 0.71428573, 0.71428573, 0.71428573, 0.71428573,
       0.71428573, 0.71428573, 0.71428573, 0.71428573, 0.71428573,
       0.71428573, 0.71428573, 0.71428573, 0.71428573, 0.71428573,
       0.71428573, 0.71428573, 0.71428573, 0.71428573, 0.71428573,
       0.71428573, 0.71428573, 0.71428573, 0.71428573], dtype=float32)

In [52]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [53]:
print("Facial Features:\n{}".format('-'*30))
results = confusion_matrix(labels_facial_feature_np, y_preds_ff)
print("Confusion Matrix:\n",results)
print(classification_report(labels_facial_feature_np, y_preds_ff))
print("Accuracy: {}".format(accuracy_score(labels_facial_feature_np, y_preds_ff)))

Facial Features:
------------------------------
Confusion Matrix:
 [[614   5]
 [ 14 678]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       619
           1       0.99      0.98      0.99       692

    accuracy                           0.99      1311
   macro avg       0.99      0.99      0.99      1311
weighted avg       0.99      0.99      0.99      1311

Accuracy: 0.9855072463768116


In [54]:
print("Facial Features:\n{}".format('-'*30))
results = confusion_matrix(labels_brain_feature_np, y_preds_bf)
print("Confusion Matrix:\n",results)
print(classification_report(labels_brain_feature_np, y_preds_bf))
print("Accuracy: {}".format(accuracy_score(labels_brain_feature_np, y_preds_bf)))

Facial Features:
------------------------------
Confusion Matrix:
 [[687  20]
 [  2 602]]
              precision    recall  f1-score   support

           0       1.00      0.97      0.98       707
           1       0.97      1.00      0.98       604

    accuracy                           0.98      1311
   macro avg       0.98      0.98      0.98      1311
weighted avg       0.98      0.98      0.98      1311

Accuracy: 0.9832189168573608
